In [18]:
import pandas as pd
import swifter

info = "info_CUB.csv"
df = pd.read_csv(info, sep=";")

def read_text(path):
    with open(path, "r") as f:
        text = f.read()
    return text

df["text"] = df["text"].swifter.apply(read_text)

label2id = {k: v for v, k in enumerate(df["label"].unique())}
id2label = {v: k for k, v in label2id.items()}
df["label"] = df["label"].map(label2id)

columns = ["image", "text", "label"]
data_train = df[df["train"] == 1][columns]
data_test = df[df["train"] == 0][columns]

Pandas Apply:   0%|          | 0/11788 [00:00<?, ?it/s]

In [19]:
# Get text from the model
from transformers import AutoModel, AutoTokenizer, AutoFeatureExtractor
from PIL import Image

# Load models gpu 0
model_nlp = AutoModel.from_pretrained("roberta-base-finetuned-CUB")
tokenizer_nlp = AutoTokenizer.from_pretrained("roberta-base-finetuned-CUB")

# Get images from the model
feature_extractor_img = AutoFeatureExtractor.from_pretrained("vit-large-patch32-384_finetuning")
model_img = AutoModel.from_pretrained("vit-large-patch32-384_finetuning")


Some weights of the model checkpoint at roberta-base-finetuned-CUB were not used when initializing RobertaModel: ['classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base-finetuned-CUB and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights

In [20]:
import numpy as np
def get_image_vector(image, model=model_img, feature_extractor=feature_extractor_img):
    try:
        # Get last layer of the model
        image = Image.open(image)
        inputs = feature_extractor(images=image, return_tensors="pt")
        outputs = model(**inputs)
        embeddings = outputs.pooler_output.detach().numpy()
        return embeddings
    except:
        print("Error with image", image)
        return np.nan


def get_text_vector(text, model=model_nlp, tokenizer=tokenizer_nlp):
    #return np.random.rand(100)
    inputs = tokenizer_nlp(text, return_tensors="pt")
    outputs = model_nlp(**inputs)
    embeddings = outputs.pooler_output.detach().numpy()
    return embeddings

data_train["v_text"] = data_train["text"].apply(get_text_vector)
data_train["v_image"] = data_train["image"].apply(get_image_vector)

data_test["v_text"] = data_test["text"].apply(get_text_vector)
data_test["v_image"] = data_test["image"].apply(get_image_vector)
# def get_data_vector(image,text):
#     try:
#         image_v = get_image_vector(image)
#         text_v = get_text_vector(text)
#         # Join two torch.Tensor in one
#         v =  np.concatenate((image_v, text_v), axis=1)
#         # Tranpose to get the right shape
#         return v
#     except:
#         return np.nan



Error with image <PIL.JpegImagePlugin.JpegImageFile image mode=L size=500x281 at 0x7FA504000100>
Error with image <PIL.JpegImagePlugin.JpegImageFile image mode=L size=500x326 at 0x7FA504000EE0>
Error with image <PIL.JpegImagePlugin.JpegImageFile image mode=L size=500x500 at 0x7FA504000EE0>
Error with image <PIL.JpegImagePlugin.JpegImageFile image mode=L size=500x398 at 0x7FA504001390>
Error with image <PIL.JpegImagePlugin.JpegImageFile image mode=L size=500x333 at 0x7FA504001030>
Error with image <PIL.JpegImagePlugin.JpegImageFile image mode=L size=484x500 at 0x7FA5040014E0>
Error with image <PIL.JpegImagePlugin.JpegImageFile image mode=L size=500x375 at 0x7FA504001090>
Error with image <PIL.JpegImagePlugin.JpegImageFile image mode=L size=485x500 at 0x7FA504001120>


In [21]:
data_train.to_pickle("vectors_roberta_vit/data_train.pkl")
data_test.to_pickle("vectors_roberta_vit/data_test.pkl")

In [22]:
import pandas as pd 
data_train = pd.read_pickle("vectors_roberta_vit/data_train.pkl")
data_test = pd.read_pickle("vectors_roberta_vit/data_test.pkl")

data_train = data_train.dropna().reset_index(drop=True)
data_test = data_test.dropna().reset_index(drop=True)

In [23]:
print(data_train.shape)
print(data_test.shape)

(5965, 5)
(5815, 5)


In [24]:
# data_train["vector"] = data_train.apply(lambda x: get_data_vector(x["image"], x["text"]), axis=1)
# data_test["vector"] = data_test.apply(lambda x: get_data_vector(x["image"], x["text"]), axis=1)


In [25]:
data_train["v_image"].apply(lambda x: x.shape).value_counts()
data_train["v_text"].apply(lambda x: x.shape).value_counts()

(1, 768)    5965
Name: v_text, dtype: int64

In [26]:
data_train["vector"] = data_train.apply(lambda x: np.concatenate((x["v_image"], x["v_text"]), axis=1), axis=1)
data_test["vector"] = data_test.apply(lambda x: np.concatenate((x["v_image"], x["v_text"]), axis=1), axis=1)

In [27]:
data_train

,image,text,label,v_text,v_image,vector
0,CUB/images/001.Black_footed_Albatross/Black_Fo...,a large brown bird with a long and sharply tur...,0,"[[-0.4494482, -0.14032204, -0.051117722, -0.37...","[[0.24670544, 0.016368864, -0.35779393, -0.405...","[[0.24670544, 0.016368864, -0.35779393, -0.405..."
1,CUB/images/003.Sooty_Albatross/Sooty_Albatross...,a large sized bird with brown tones in the win...,1,"[[-0.49942705, -0.45266795, -0.07493907, -0.45...","[[-0.77163243, 0.14098488, 0.5532929, 0.250189...","[[-0.77163243, 0.14098488, 0.5532929, 0.250189..."
2,CUB/images/004.Groove_billed_Ani/Groove_Billed...,the bird has a very tiny black bill as well as...,2,"[[0.53590196, 0.07426909, 0.30264488, -0.65106...","[[-0.24587081, 0.5262232, -0.60334283, -0.6706...","[[-0.24587081, 0.5262232, -0.60334283, -0.6706..."
3,CUB/images/005.Crested_Auklet/Crested_Auklet_0...,this bird is almost all light gray with a whit...,3,"[[-0.03630162, -0.0089132665, 0.18595152, -0.1...","[[-0.62728, -0.4452654, -0.7264618, 0.33525065...","[[-0.62728, -0.4452654, -0.7264618, 0.33525065..."
4,CUB/images/007.Parakeet_Auklet/Parakeet_Auklet...,the bird has a white eyering and a white breas...,5,"[[0.03143559, 0.49735644, 0.3504617, 0.0174330...","[[0.6704706, 0.12495472, -0.5906075, -0.305272...","[[0.6704706, 0.12495472, -0.5906075, -0.305272..."
...,...,...,...,...,...,...
5960,CUB/images/118.House_Sparrow/House_Sparrow_014...,"small roundish bird, with an off white abdomen...",72,"[[0.40644965, -0.22581306, 0.034566283, 0.0558...","[[-0.67271984, -0.50511837, -0.07102936, 0.348...","[[-0.67271984, -0.50511837, -0.07102936, 0.348..."
5961,CUB/images/197.Marsh_Wren/Marsh_Wren_0145_1881...,this is a bird with a white belly and a long o...,153,"[[0.23964927, -0.3913797, -0.1839547, 0.113428...","[[0.33722794, 0.030788623, -0.22617392, 0.0269...","[[0.33722794, 0.030788623, -0.22617392, 0.0269..."
5962,CUB/images/062.Herring_Gull/Herring_Gull_0143_...,the bird is white with black tail feathers tha...,35,"[[0.3628128, -0.38673288, -0.10301982, -0.1289...","[[0.16473852, -0.5832121, -0.5755116, -0.50059...","[[0.16473852, -0.5832121, -0.5755116, -0.50059..."
5963,CUB/images/066.Western_Gull/Western_Gull_0143_...,"this bird has a white head with yellow bill, i...",163,"[[0.11574447, -0.50187784, 0.7015084, 0.200828...","[[0.175866, -0.7046889, 0.064728186, -0.074174...","[[0.175866, -0.7046889, 0.064728186, -0.074174..."


In [28]:
# Convert vector to a list of numbers


data_train["vector"] = data_train["vector"].apply(lambda x: x.tolist()[0])
data_test["vector"] = data_test["vector"].apply(lambda x: x.tolist()[0])

In [29]:
# MLP sklearn model with 4 hidden layers
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

X_train = data_train["vector"].tolist()
y_train = data_train["label"].tolist()

X_test = data_test["vector"].tolist()
y_test = data_test["label"].tolist()




In [37]:
# Grid search to find the best parameters for the model
from sklearn.model_selection import GridSearchCV

params = {
    "hidden_layer_sizes": [(1000, 600), (1000, 700),(1000, 500)],
    "activation": ["tanh", "relu"],
    "solver": ["adam", "sgd"],
    "alpha": [0.05, 0.01, 0.03, 0.1],
    "learning_rate": ["adaptive"],
    
}

mlp = MLPClassifier(max_iter=20, random_state=42, verbose=False, early_stopping=True, validation_fraction=0.1)

# Custom scorer    

clf = GridSearchCV(mlp, params, n_jobs=4, verbose=10, cv=2, scoring="accuracy")
clf.fit(X_train, y_train)

print(clf.best_params_)

Fitting 2 folds for each of 48 candidates, totalling 96 fits
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKEN

/home/jlortiz/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV 2/2; 2/48] END activation=tanh, alpha=0.05, hidden_layer_sizes=(1000, 600), learning_rate=adaptive, solver=sgd;, score=0.955 total time=  46.8s
[CV 2/2; 3/48] START activation=tanh, alpha=0.05, hidden_layer_sizes=(1000, 700), learning_rate=adaptive, solver=adam


/home/jlortiz/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV 1/2; 2/48] END activation=tanh, alpha=0.05, hidden_layer_sizes=(1000, 600), learning_rate=adaptive, solver=sgd;, score=0.961 total time=  50.0s
[CV 1/2; 4/48] START activation=tanh, alpha=0.05, hidden_layer_sizes=(1000, 700), learning_rate=adaptive, solver=sgd
[CV 2/2; 1/48] END activation=tanh, alpha=0.05, hidden_layer_sizes=(1000, 600), learning_rate=adaptive, solver=adam;, score=0.982 total time=  54.0s
[CV 2/2; 4/48] START activation=tanh, alpha=0.05, hidden_layer_sizes=(1000, 700), learning_rate=adaptive, solver=sgd
[CV 1/2; 3/48] END activation=tanh, alpha=0.05, hidden_layer_sizes=(1000, 700), learning_rate=adaptive, solver=adam;, score=0.984 total time=  45.4s
[CV 1/2; 5/48] START activation=tanh, alpha=0.05, hidden_layer_sizes=(1000, 500), learning_rate=adaptive, solver=adam
[CV 2/2; 3/48] END activation=tanh, alpha=0.05, hidden_layer_sizes=(1000, 700), learning_rate=adaptive, solver=adam;, score=0.981 total time=  51.7s
[CV 2/2; 5/48] START activation=tanh, alpha=0.05, hid

...

[CV 1/2; 47/48] END activation=relu, alpha=0.1, hidden_layer_sizes=(1000, 500), learning_rate=adaptive, solver=adam;, score=0.983 total time=  50.8s
[CV 2/2; 47/48] END activation=relu, alpha=0.1, hidden_layer_sizes=(1000, 500), learning_rate=adaptive, solver=adam;, score=0.972 total time=  28.2s


/home/jlortiz/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV 1/2; 48/48] END activation=relu, alpha=0.1, hidden_layer_sizes=(1000, 500), learning_rate=adaptive, solver=sgd;, score=0.878 total time=  35.8s


/home/jlortiz/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


[CV 2/2; 48/48] END activation=relu, alpha=0.1, hidden_layer_sizes=(1000, 500), learning_rate=adaptive, solver=sgd;, score=0.886 total time=  31.1s
{'activation': 'tanh', 'alpha': 0.03, 'hidden_layer_sizes': (1000, 500), 'learning_rate': 'adaptive', 'solver': 'adam'}


In [36]:
print(clf.best_params_)
mlp = MLPClassifier(**clf.best_params_, max_iter=20, random_state=42, verbose=False, early_stopping=True, validation_fraction=0.1)
mlp.fit(X_train, y_train)
y_pred = mlp.predict(X_test)
print("Accuracy: ", round(accuracy_score(y_test, y_pred), 2))

#{'activation': 'tanh', 'alpha': 0.01, 'hidden_layer_sizes': (1000, 500), 'learning_rate': 'adaptive', 'solver': 'adam'}


{'activation': 'tanh', 'alpha': 0.05, 'hidden_layer_sizes': (1000, 500), 'learning_rate': 'adaptive', 'solver': 'adam'}
Accuracy:  0.86
